In [ ]:
import csv
from dataclasses import dataclass, field, asdict
from datetime import datetime
from typing import List, Optional
from collections import Counter

@dataclass(order=True)
class Item:
    category: str
    value: float
    
    name: str = field(compare=False)
    quantity: int = field(compare=False)
    condition: str = field(compare=False)
    location: str = field(compare=False)
    
    added_at: datetime = field(default_factory=datetime.now, compare=False)

    def total_value(self) -> float:
        return self.quantity * self.value

    def __str__(self) -> str:
        return (f"[{self.category}] {self.name} ({self.quantity} шт.) — "
                f"{self.value} грн/шт, стан: {self.condition}")


@dataclass
class Inventory:
    items: List[Item] = field(default_factory=list)

    def add_item(self, item: Item):
        self.items.append(item)
        print(f"Додано: {item.name}")

    def remove_item(self, name: str):
        initial_count = len(self.items)
        self.items = [item for item in self.items if item.name != name]
        if len(self.items) < initial_count:
            print(f"Видалено: {name}")
        else:
            print(f"Предмет '{name}' не знайдено.")

    def find_by_category(self, category: str) -> List[Item]:
        return [item for item in self.items if item.category == category]

    def total_inventory_value(self) -> float:
        return sum(item.total_value() for item in self.items)

    def export_summary(self):
        categories = [item.category for item in self.items]

        summary = Counter(categories)
        print("\n--- ЗВІТ ПО КАТЕГОРІЯХ ---")
        for cat, count in summary.items():
            print(f"{cat}: {count} позицій")
        print("--------------------------")

    def filter_items(self, **kwargs) -> List[Item]:
        result = []
        for item in self.items:
            match = True
            for key, value in kwargs.items():

                if getattr(item, key, None) != value:
                    match = False
                    break
            if match:
                result.append(item)
        return result

    def sort_items(self):
        self.items.sort()
        print("Інвентар відсортовано (Категорія -> Ціна).")

    def save_to_csv(self, filename: str):
        if not self.items:
            print("Інвентар порожній, нічого зберігати.")
            return

        with open(filename, mode='w', newline='', encoding='utf-8') as f:

            fieldnames = [f.name for f in dataclasses.fields(Item)]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            writer.writeheader()
            for item in self.items:
                data = asdict(item)
                writer.writerow(data)
        print(f"Дані збережено у {filename}")

    def load_from_csv(self, filename: str):
        try:
            with open(filename, mode='r', encoding='utf-8') as f:
                reader = csv.DictReader(f)
                self.items = []
                for row in reader:
                    item = Item(
                        category=row['category'],
                        value=float(row['value']),
                        name=row['name'],
                        quantity=int(row['quantity']),
                        condition=row['condition'],
                        location=row['location']
                    )
                    if 'added_at' in row:
                        item.added_at = datetime.fromisoformat(row['added_at'])
                    self.items.append(item)
            print(f"Дані завантажено з {filename}")
        except FileNotFoundError:
            print("Файл не знайдено.")


import dataclasses 

if __name__ == "__main__":
    my_inventory = Inventory()

    item1 = Item("інструменти", 150.0, "Гайковий ключ", 3, "уживаний", "гараж")
    item2 = Item("електроніка", 5000.0, "Старий монітор", 1, "новий", "комора")
    item3 = Item("металобрухт", 50.0, "Мідний дріт", 10, "зламаний", "сарай")
    item4 = Item("інструменти", 4500.0, "Дриль", 1, "новий", "гараж")

    my_inventory.add_item(item1)
    my_inventory.add_item(item2)
    my_inventory.add_item(item3)
    my_inventory.add_item(item4)


    print(f"\nТест str: {item1}")
    print(f"Вартість усіх ключів: {item1.total_value()} грн")

    print("\n--- До сортування ---")
    for i in my_inventory.items:
         print(i)
    
    my_inventory.sort_items()
    
    print("\n--- Після сортування (Електроніка -> Інструменти[дешеві] -> Інструменти[дорогі] -> Метал) ---")
    for i in my_inventory.items:
         print(i)

    my_inventory.export_summary()
    print(f"Загальна вартість всього мотлоху: {my_inventory.total_inventory_value()} грн")

    print("\n--- Шукаємо нові речі ---")
    new_items = my_inventory.filter_items(condition="новий")
    for i in new_items:
         print(i)

    my_inventory.save_to_csv("my_stuff.csv")
    
    my_inventory.items = []
    print("\nІнвентар очищено. Завантажуємо з файлу...")
    my_inventory.load_from_csv("my_stuff.csv")
    for i in my_inventory.items:
         print(i)

Додано: Гайковий ключ
Додано: Старий монітор
Додано: Мідний дріт
Додано: Дриль

Тест str: [інструменти] Гайковий ключ (3 шт.) — 150.0 грн/шт, стан: уживаний
Вартість усіх ключів: 450.0 грн

--- До сортування ---
[інструменти] Гайковий ключ (3 шт.) — 150.0 грн/шт, стан: уживаний
[електроніка] Старий монітор (1 шт.) — 5000.0 грн/шт, стан: новий
[металобрухт] Мідний дріт (10 шт.) — 50.0 грн/шт, стан: зламаний
[інструменти] Дриль (1 шт.) — 4500.0 грн/шт, стан: новий
Інвентар відсортовано (Категорія -> Ціна).

--- Після сортування (Електроніка -> Інструменти[дешеві] -> Інструменти[дорогі] -> Метал) ---
[електроніка] Старий монітор (1 шт.) — 5000.0 грн/шт, стан: новий
[металобрухт] Мідний дріт (10 шт.) — 50.0 грн/шт, стан: зламаний
[інструменти] Гайковий ключ (3 шт.) — 150.0 грн/шт, стан: уживаний
[інструменти] Дриль (1 шт.) — 4500.0 грн/шт, стан: новий

--- ЗВІТ ПО КАТЕГОРІЯХ ---
електроніка: 1 позицій
металобрухт: 1 позицій
інструменти: 2 позицій
--------------------------
Загальна вартіст